In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from tabulate import tabulate
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn.metrics import mean_squared_error, mean_absolute_error

## **Dataset**

In [ ]:
books=pd.read_csv("/content/drive/MyDrive/CSE 488/Book_Recommendation_System/BX_Books.csv", delimiter=";", encoding="latin1")
ratings=pd.read_csv("/content/drive/MyDrive/CSE 488/Book_Recommendation_System/BX-Book-Ratings.csv", delimiter=";", encoding="latin1")
users=pd.read_csv("/content/drive/MyDrive/CSE 488/Book_Recommendation_System/BX-Users.csv", delimiter=";", encoding="latin1")

# **Books-preprocessing**

In [ ]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [ ]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271379 non-null  object
 1   Book-Title           271379 non-null  object
 2   Book-Author          271378 non-null  object
 3   Year-Of-Publication  271379 non-null  int64 
 4   Publisher            271377 non-null  object
 5   Image-URL-S          271379 non-null  object
 6   Image-URL-M          271379 non-null  object
 7   Image-URL-L          271379 non-null  object
dtypes: int64(1), object(7)
memory usage: 16.6+ MB


In [ ]:
books.shape

(271379, 8)

In [ ]:
books.dropna(inplace=True)

In [ ]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
Image-URL-S            0
Image-URL-M            0
Image-URL-L            0
dtype: int64

In [ ]:
books.duplicated().sum()

0

# **Ratings-preprocessing**

In [ ]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [ ]:
ratings.shape

(1149780, 3)

In [ ]:
ratings.duplicated().sum()

0

# **users-preprocessing**

In [ ]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [ ]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [ ]:
users.shape

(278858, 3)

In [ ]:
users.dropna(inplace=True)

In [ ]:
users.duplicated().sum()

0

# combine the rating data with the total rating count data to determine which books are popular while excluding less popular ones.

In [ ]:
rating_with_title = ratings.merge(books,on="ISBN")
rating_with_title

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
...,...,...,...,...,...,...,...,...,...,...
1031167,276688,0517145553,0,Mostly Harmless,Douglas Adams,1995,Random House Value Pub,http://images.amazon.com/images/P/0517145553.0...,http://images.amazon.com/images/P/0517145553.0...,http://images.amazon.com/images/P/0517145553.0...
1031168,276688,1575660792,7,Gray Matter,Shirley Kennett,1996,Kensington Publishing Corporation,http://images.amazon.com/images/P/1575660792.0...,http://images.amazon.com/images/P/1575660792.0...,http://images.amazon.com/images/P/1575660792.0...
1031169,276690,0590907301,0,Triplet Trouble and the Class Trip (Triplet Tr...,Debbie Dadey,1997,Apple,http://images.amazon.com/images/P/0590907301.0...,http://images.amazon.com/images/P/0590907301.0...,http://images.amazon.com/images/P/0590907301.0...
1031170,276704,0679752714,0,A Desert of Pure Feeling (Vintage Contemporaries),Judith Freeman,1997,Vintage Books USA,http://images.amazon.com/images/P/0679752714.0...,http://images.amazon.com/images/P/0679752714.0...,http://images.amazon.com/images/P/0679752714.0...


In [ ]:
rating_count=rating_with_title.groupby("Book-Title").count()['Book-Rating'].reset_index()
rating_count.rename(columns={"Book-Rating":"Total-rating_Count"},inplace=True)
rating_count

,Book-Title,Total-rating_Count
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241083,Ã?Â?lpiraten.,2
241084,Ã?Â?rger mit Produkt X. Roman.,4
241085,Ã?Â?sterlich leben.,1
241086,Ã?Â?stlich der Berge.,3


In [ ]:
avg_rating_count=rating_with_title.groupby("Book-Title").mean()["Book-Rating"].reset_index()
avg_rating_count.rename(columns={"Book-Rating":"Avg_rating_Count"},inplace=True)
avg_rating_count

<ipython-input-24-a56f2de215ca>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  avg_rating_count=rating_with_title.groupby("Book-Title").mean()["Book-Rating"].reset_index()


,Book-Title,Avg_rating_Count
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241083,Ã?Â?lpiraten.,0.000000
241084,Ã?Â?rger mit Produkt X. Roman.,5.250000
241085,Ã?Â?sterlich leben.,7.000000
241086,Ã?Â?stlich der Berge.,2.666667


In [ ]:
popular_df = rating_count.merge(avg_rating_count,on="Book-Title")
popular_df

,Book-Title,Total-rating_Count,Avg_rating_Count
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241083,Ã?Â?lpiraten.,2,0.000000
241084,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
241085,Ã?Â?sterlich leben.,1,7.000000
241086,Ã?Â?stlich der Berge.,3,2.666667


In [ ]:
popular_books = popular_df[popular_df["Total-rating_Count"] >= 250].sort_values("Avg_rating_Count", ascending=False).head(50)

In [ ]:
popular_books.head(50)

,Book-Title,Total-rating_Count,Avg_rating_Count
80437,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804
80425,Harry Potter and the Goblet of Fire (Book 4),387,5.824289
80444,Harry Potter and the Sorcerer's Stone (Book 1),278,5.737410
80429,Harry Potter and the Order of the Phoenix (Boo...,347,5.501441
80417,Harry Potter and the Chamber of Secrets (Book 2),556,5.183453
191621,The Hobbit : The Enchanting Prelude to The Lor...,281,5.007117
187386,The Fellowship of the Ring (The Lord of the Ri...,368,4.948370
80448,Harry Potter and the Sorcerer's Stone (Harry P...,575,4.895652
211394,"The Two Towers (The Lord of the Rings, Part 2)",260,4.880769
219752,To Kill a Mockingbird,510,4.700000


# **Taking user id as input and show the top-10 recommended books for the user.**

In [ ]:
reader = rating_with_title.groupby('User-ID').count()['Book-Rating'] > 200
most_read_users = reader[reader].index

In [ ]:
filtered_rating = rating_with_title[rating_with_title['User-ID'].isin(most_read_users)]

In [ ]:
rated_book = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books = rated_book[rated_book].index

In [ ]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

In [ ]:
pt = final_ratings.pivot_table(index='User-ID',columns='Book-Title',values='Book-Rating')

In [ ]:
pt.fillna(0,inplace=True)

In [ ]:
pt

Book-Title,1984,1st to Die: A Novel,2nd Chance,4 Blondes,A Bend in the Road,A Case of Need,"A Child Called \It\"": One Child's Courage to Survive""",A Civil Action,A Day Late and a Dollar Short,A Fine Balance,...,Winter Solstice,Wish You Well,Without Remorse,"Wizard and Glass (The Dark Tower, Book 4)",Wuthering Heights,Year of Wonders,You Belong To Me,Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,Zoya,"\O\"" Is for Outlaw"""
User-ID,,,,,,,,,,,,,,,,,,,,,
254,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2276,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2766,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0
3363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274308,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
275970,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
277427,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
similarity_scores = cosine_similarity(pt)

In [ ]:
similarity_scores.shape

(810, 810)

In [ ]:
def recommend_book_by_usedId(user_id):
    if user_id not in pt.index:
        return f"User {user_id} not found in the dataset."

    index = np.where(pt.index == user_id)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])), key=lambda x: x[1], reverse=True)[1:11]

    data = []
    for i in similar_items:
        temp_df = books[books['Book-Title'] == pt.columns[i[0]]]
        book_title = temp_df['Book-Title'].values[0]
        book_author = temp_df['Book-Author'].values[0]
        data.append({'Book-Title': book_title, 'Book-Author': book_author})

    return data

In [ ]:
recommend_book_by_usedId (17)

'User 17 not found in the dataset.'

In [ ]:
recommend_book_by_usedId (254)

[{'Book-Title': 'The Perfect Storm : A True Story of Men Against the Sea',
  'Book-Author': 'Sebastian Junger'},
 {'Book-Title': 'The Clinic (Alex Delaware Novels (Paperback))',
  'Book-Author': 'Jonathan Kellerman'},
 {'Book-Title': 'The Mulberry Tree', 'Book-Author': 'Jude Deveraux'},
 {'Book-Title': 'No Safe Place', 'Book-Author': 'RICHARD NORTH PATTERSON'},
 {'Book-Title': 'The Bourne Identity', 'Book-Author': 'Robert Ludlum'},
 {'Book-Title': 'Intensity', 'Book-Author': 'DEAN KOONTZ'},
 {'Book-Title': 'Taltos: Lives of the Mayfair Witches',
  'Book-Author': 'Anne Rice'},
 {'Book-Title': 'Invasion', 'Book-Author': 'Robin Cook'},
 {'Book-Title': 'The Alienist', 'Book-Author': 'Caleb Carr'},
 {'Book-Title': "Fortune's Rocks: A Novel", 'Book-Author': 'Anita Shreve'}]

In [ ]:
recommend_book_by_usedId (253)

'User 253 not found in the dataset.'

Taking user id and ISBN as input and show
the probable rating of that book by that user.

In [ ]:
selected_user_id = rating_with_title['User-ID'].unique()[1:500]

selected_ratings = rating_with_title[rating_with_title['User-ID'].isin(selected_user_id)]

pt2 = selected_ratings.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating')

pt2.fillna(0, inplace=True)

In [ ]:
pt2

ISBN,0001047868,000104799X,000123207X,0001232088,0001821326,0001837397,0001841572,0001842749,0001848445,0001900277,...,B00005UMK6,B0000640CT,B000078UH8,B00007MF56,B00009EF82,B0000C37EK,B0000DYXID,B0000E63CJ,B00016560C,B0001I1KOG
User-ID,,,,,,,,,,,,,,,,,,,,,
243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
645,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1660,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276744,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
276746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
277427,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
item_similarity_scores = cosine_similarity(pt2)

In [ ]:
item_similarity_scores.shape

(499, 499)

In [ ]:
selected_user_ids = rating_with_title['User-ID'].unique()[:500]

selected_ratings = rating_with_title[rating_with_title['User-ID'].isin(selected_user_ids)]

pvt2 = selected_ratings.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating')

pvt2.fillna(0, inplace=True)


interactions_sparse = csr_matrix(pvt2.values)

model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_jobs=-1)
model_knn.fit(interactions_sparse)


def get_book_recommendations(user_id, k=10):
    user_idx = pvt2.index.tolist().index(user_id)
    distances, indices = model_knn.kneighbors(interactions_sparse[user_idx], n_neighbors=k+1)

    similar_user_ids = [pvt2.index[i] for i in indices.flatten()[1:]]
    similar_users_ratings = pvt2.loc[similar_user_ids].mean(axis=0)
    top_isbns = similar_users_ratings.nlargest(10).index
    top_books = books.loc[books['ISBN'].isin(top_isbns)]


    top_books = top_books.drop_duplicates(subset=['ISBN'])

    return top_books[['ISBN', 'Book-Title']]


def predict_book_rating(user_id, isbn, k=10):
    try:
        user_idx = pvt2.index.tolist().index(user_id)
        book_idx = pvt2.columns.tolist().index(isbn)
    except ValueError:
        return "User ID or ISBN not found in the dataset."

    distances, indices = model_knn.kneighbors(interactions_sparse[user_idx], n_neighbors=k+1)
    distances = distances.flatten()[1:]
    indices = indices.flatten()[1:]

    neighbors_ratings = pvt2.iloc[indices, book_idx]
    weighted_ratings = neighbors_ratings / distances
    predicted_rating = weighted_ratings.sum() / (1 / distances).sum()
    actual_rating = pvt2.iloc[user_idx, book_idx]

    return predicted_rating, actual_rating

In [ ]:
pvt2.head(10)

ISBN,0001047868,000104799X,000123207X,0001232088,0001821326,0001837397,0001841572,0001842749,0001848445,0001900277,...,B00005UMK6,B0000640CT,B000078UH8,B00007MF56,B00009EF82,B0000C37EK,B0000DYXID,B0000E63CJ,B00016560C,B0001I1KOG
User-ID,,,,,,,,,,,,,,,,,,,,,
243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
645,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1660,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2288,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4551,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
user_id = 2010
isbn = '000104799X'

predicted_rating, actual_rating = predict_book_rating(user_id, isbn)
print(f"Predicted rating User {user_id} in Book ISBN {isbn} is: {predicted_rating}")


Predicted rating User 2010 in Book ISBN 000104799X is: 0.0
